In [15]:
import json
import logging
import time

import requests


DATA_URL = "https://www.tripadvisor.es/data/graphql/ids"
REQ_LIMIT = 40
LOCATION_ID = 191050
PAYLOAD =  [
    {
        "variables": {
            "locationId": LOCATION_ID,
            "albumId": -160,
            "subAlbumId": -160,
            "client": "ar",
            "dataStrategy": "ar",
            "filter": {
                "mediaGroup":"ALL_INCLUDING_RESTRICTED"
            },
            "offset": 0,
            "limit": REQ_LIMIT,
        },
        "extensions": {
            "preRegisteredQueryId":"2d46abde60a014b0"
        }
    }
]
MAX_ERRORS = 5
HEADERS = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
    "Accept-Encoding": "gzip, deflate",
    "Content-Type": "application/json",
    "Connection": "keep-alive",
}

In [16]:
offset = 0
consecutive_errors = 0


def get_media_list(response) -> list:

    return response[0]["data"]["mediaAlbumPage"]["mediaList"]

while True:
    if consecutive_errors > MAX_ERRORS:
        logging.error("Max. errors reached. Breaking.")
        break

    PAYLOAD[0]["variables"]["offset"] = offset
    time.sleep(3)

    try:
        response = requests.post(
            DATA_URL,
            headers=HEADERS,
            data=json.dumps(PAYLOAD),
            timeout=10,
        ).json()
    except Exception as e:
        logging.error(e)
        consecutive_errors += 1
        continue

    consecutive_errors = 0
    with open(f"output/{LOCATION_ID}_{offset}.json", "w") as f:
        f.write(json.dumps(response))

    if not get_media_list(response):
        break

    offset += 20

ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:HTTPSConnectionPool(host='www.tripadvisor.es', port=443): Read timed out. (read timeout=5)
ERROR:root:Max. errors reached. Breaking.
